In [1]:
# import packages
import torch
import torch.nn as nn
from torchvision import datasets, transforms

# Entendendo o Código de Classificação de Dígitos (MNIST)

Este bloco de código demonstra como carregar uma imagem do famoso dataset MNIST e passá-la por uma Rede Neural simples (MLP) para prever qual dígito está escrito à mão.



### 1. Preparação e Carregamento dos Dados
* **`transform`**: Converte as imagens baixadas para o formato "Tensor" do PyTorch.
* **`mnist_test`**: Baixa e carrega o conjunto de dados de teste (validação).
* **`image, label`**: Extrai a primeira imagem do dataset (índice 0) e seu rótulo real (a resposta correta).

### 2. Achatamento da Imagem (Flattening)
* As imagens originais têm 28x28 pixels. Redes neurais simples precisam de um vetor unidimensional (uma linha reta de dados) para funcionar.
* O comando `view(1, 784)` "achata" a grade da imagem em um único vetor de 784 números (28 * 28 = 784). O `1` no início indica que estamos processando um lote (*batch*) de apenas 1 imagem.

### 3. Construção da Rede Neural (Modelo)

* **`nn.Sequential`**: Cria a rede empilhando camadas de forma sequencial.
* **`nn.Linear(784, 128)`**: Camada de entrada. Recebe os 784 pixels e os conecta a 128 "neurônios" ocultos.
* **`nn.ReLU()`**: Função de ativação. Adiciona não-linearidade ao modelo, permitindo que ele aprenda padrões complexos.
* **`nn.Linear(128, 10)`**: Camada de saída. Reduz os 128 sinais para 10 saídas, representando as 10 classes possíveis (os dígitos de 0 a 9).

### 4. Fazendo a Previsão (Inferência)
* **`torch.no_grad()`**: Desativa o cálculo de gradientes (derivadas) para economizar memória e processamento, já que estamos apenas testando, e não treinando.
* **`logits`**: A saída "bruta" do modelo com valores arbitrários para cada classe.
* **`torch.softmax(...)`**: Converte os valores brutos em probabilidades que somam 1 (100%).
* **`torch.argmax(...)`**: Olha para as probabilidades e escolhe o índice com o maior valor. Esta é a previsão final da rede!

> **⚠️ Nota:** Como este modelo acabou de ser criado e ainda **não foi treinado**, seus pesos internos são completamente aleatórios. A previsão inicial será apenas um "chute".

In [2]:
# 1. Digit Classification

transform = transforms.Compose([transforms.ToTensor()])
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
image, label = mnist_test[0]

flat_image = image.view(1, 784)

mlp_model = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

with torch.no_grad():
    logits = mlp_model(flat_image)
    probabilities = torch.softmax(logits, dim=1)
    prediction = torch.argmax(probabilities, dim=1)

print("--- MLP MNIST CLASSIFICATION TEST ---")
print(f"Actual Label: {label}")
print(f"Output Shape: {logits.shape}") # [1, 10]
print(f"Predicted Class: {prediction.item()}")
print(f"Confidence in Prediction: {probabilities.max().item():.4f}")

100.0%
100.0%
100.0%
100.0%

--- MLP MNIST CLASSIFICATION TEST ---
Actual Label: 7
Output Shape: torch.Size([1, 10])
Predicted Class: 1
Confidence in Prediction: 0.1209


# Extraindo Mapas de Características (Feature Maps) com CNNs

Este bloco demonstra o passo fundamental das Redes Neurais Convolucionais (CNNs): aplicar filtros matemáticos em uma imagem colorida (do dataset CIFAR-10) para extrair padrões visuais, como bordas, texturas e cores.

### 1. Preparação dos Dados (Dataset CIFAR-10)
* **`CIFAR10(...)`**: Baixa o dataset CIFAR-10. Diferente do MNIST (que é preto e branco), o CIFAR-10 contém imagens coloridas de 32x32 pixels divididas em 10 classes (aviões, carros, pássaros, etc.).
* **`image, label`**: Pega a primeira imagem de teste. No PyTorch, o formato dessa imagem será `[3, 32, 32]` (3 canais de cor RGB, 32 pixels de altura, 32 pixels de largura).

### 2. Preparando o "Lote" (Batching)
* **`image.unsqueeze(0)`**: O PyTorch sempre espera processar imagens em lotes (*batches*), mesmo que seja apenas uma imagem. O comando `unsqueeze(0)` adiciona uma nova dimensão no início, transformando o formato de `[3, 32, 32]` para `[1, 3, 32, 32]` (1 imagem, 3 canais, 32x32 pixels).

### 3. A Camada Convolucional

* **`nn.Conv2d(...)`**: Cria uma camada de convolução 2D. É aqui que a mágica acontece:
    * **`in_channels=3`**: Diz à camada que a imagem de entrada tem 3 canais de cor (Vermelho, Verde, Azul).
    * **`out_channels=16`**: Define que queremos aplicar **16 filtros diferentes**. Cada filtro vai procurar um padrão visual específico e gerar o seu próprio "mapa de características" (feature map).
    * **`kernel_size=3`**: O tamanho do filtro. É uma matriz de 3x3 pixels que vai deslizar por toda a imagem fazendo cálculos matemáticos.

### 4. Extração e Resultados
* **`output = conv_layer(image_batch)`**: Passa a imagem pela camada convolucional (sem calcular gradientes com `torch.no_grad()`, pois é apenas um teste).
* **O Resultado (`output.shape`)**: O formato final gerado será `[1, 16, 30, 30]`.
    * O `1` é o lote.
    * O `16` são os novos mapas gerados (um para cada filtro).
    * O tamanho caiu de `32x32` para `30x30` porque um filtro 3x3 "perde" as bordas da imagem ao deslizar sobre ela (já que não adicionamos preenchimento extra, ou *padding*).

In [3]:
#Extracting 16 spatial feature maps from a color image.
transform = transforms.Compose([transforms.ToTensor()])
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
image, label = test_set[0]
image_batch = image.unsqueeze(0)

conv_layer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)

with torch.no_grad():
    output = conv_layer(image_batch)

print("--- CNN CIFAR-10 TEST ---")
print(f"Original Image Shape: {image.shape}")
print(f"After Conv Layer (16 filters): {output.shape}")

100.0%


--- CNN CIFAR-10 TEST ---
Original Image Shape: torch.Size([3, 32, 32])
After Conv Layer (16 filters): torch.Size([1, 16, 30, 30])


c:\Source\Repos\obimelo\Curso-MachineLearning-FCUP\.venv\Lib\site-packages\torchvision\datasets\cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


# 📈 Previsão de Séries Temporais com Redes Recorrentes (LSTM)

Este bloco de código demonstra como usar uma arquitetura **LSTM** (Long Short-Term Memory) para prever o próximo passo ($t+1$) de uma série temporal contínua com base em um histórico recente.



### 1. Criando os Dados (A Onda Senoidal)

* **`torch.linspace(0, 50, 500)`**: Cria um eixo de tempo com 500 pontos, distribuídos do 0 ao 50.
* **`series = torch.sin(t)`**: Aplica a função matemática seno sobre esses pontos. Isso gera dados sequenciais perfeitos para testarmos a capacidade de memória da rede.

### 2. A Janela Deslizante (Sliding Window)

* **`window_size = 10`**: O modelo usará os últimos 10 passos de tempo para prever o 11º.
* **`test_window = ...view(1, window_size, 1)`**: O PyTorch exige que os dados para LSTMs tenham o formato exato tridimensional de `[Batch, Sequence, Feature]`. Aqui nós moldamos nossa janela para:
    * **Batch (1):** Um lote por vez.
    * **Sequence (10):** Os 10 passos temporais.
    * **Feature (1):** Um único valor por passo (a amplitude da onda).

### 3. Construindo a Arquitetura
* **`nn.LSTM(...)`**: A camada central do modelo.
    * `input_size=1`: Recebe um valor por instante de tempo.
    * `hidden_size=32`: A LSTM processa os dados e cria uma "memória oculta" com 32 dimensões.
    * `batch_first=True`: Indica que a primeira dimensão do nosso tensor é o tamanho do lote (*Batch*).
* **`nn.Linear(32, 1)`**: A camada de saída. Ela pega a memória complexa de 32 dimensões gerada pela LSTM e a decodifica em 1 único número (a nossa previsão para o futuro).

### 4. Executando a Previsão (Inferência)
* **`with torch.no_grad():`**: Desliga o motor de aprendizado do PyTorch (cálculo de gradientes) para economizar memória durante o teste.
* **`out, _ = lstm(test_window)`**: Passamos a janela de 10 passos pela LSTM. Ela retorna o histórico de saídas (`out`).
* **`prediction = fc(out[:, -1, :])`**: Este é o truque principal! Nós fatiamos o tensor para pegar apenas a saída do **último passo** (`-1`), pois ele contém a memória acumulada de toda a sequência lida. Em seguida, passamos isso pela camada Linear para obter o valor final.

> **💡 Nota:** O valor previsto (`Model Prediction`) será aleatório comparado ao valor real (`Actual Next Value`) porque as camadas foram inicializadas com pesos randômicos e o modelo ainda não foi treinado.

In [8]:
# Predicting t+1 based on a window of 10 previous steps.
t = torch.linspace(0, 50, 500)
series = torch.sin(t)

window_size = 10
test_window = series[:window_size].view(1, window_size, 1) # [Batch, Seq, Feature]

lstm = nn.LSTM(input_size=1, hidden_size=32, batch_first=True)
fc = nn.Linear(32, 1)

with torch.no_grad():
    out, _ = lstm(test_window)
    prediction = fc(out[:, -1, :])

print("\n--- RNN SINE WAVE TEST ---")
print(f"Input Sequence (first 3): {test_window.flatten()[:3].tolist()}")
print(f"Actual Next Value: {series[window_size].item():.4f}")
print(f"Model Prediction: {prediction.item():.4f}")


--- RNN SINE WAVE TEST ---
Input Sequence (first 3): [0.0, 0.10003281384706497, 0.19906212389469147]
Actual Next Value: 0.8426
Model Prediction: -0.1149


# Processamento de Texto com Transformers (Mecanismo de Atenção)

Este bloco de código demonstra como usar uma camada de **Transformer** para criar *embeddings* (vetores matemáticos) cientes do contexto para uma frase. Diferente de modelos mais antigos, o Transformer olha para todas as palavras da frase ao mesmo tempo para entender como elas se relacionam.

### 1. Tokenização e Vocabulário
* **`sentence` e `vocab`**: Computadores não leem texto, eles leem números. Aqui criamos um pequeno dicionário (vocabulário) que mapeia cada palavra da nossa frase para um número inteiro (ID). Por exemplo, "Deep" vira `0`, "learning" vira `1`.
* **`tokens = torch.tensor(...).unsqueeze(1)`**: Converte a frase em um tensor de IDs. O `unsqueeze(1)` formata os dados no formato padrão que o PyTorch espera para Transformers que não usam `batch_first=True`: `[Sequence Length, Batch]`. O resultado é `[4, 1]` (4 palavras, 1 lote).

### 2. A Camada de Embedding (Vetores de Palavras)

* **`embed = nn.Embedding(num_embeddings=10, embedding_dim=16)`**: Esta camada transforma os IDs inteiros (0, 1, 2, 3) em vetores densos com números decimais.
    * `num_embeddings=10`: O tamanho máximo do nosso vocabulário (arbitrário aqui, mas suficiente para nossas 4 palavras).
    * `embedding_dim=16`: Cada palavra será representada por uma lista de 16 números.

### 3. A Camada Transformer (Self-Attention)

* **`tf_layer = nn.TransformerEncoderLayer(...)`**: O coração do modelo.
    * `d_model=16`: O tamanho da entrada (precisa bater exatamente com os 16 do embedding).
    * `nhead=4`: Define 4 "cabeças de atenção" (*Multi-Head Attention*). Isso significa que a rede tem 4 formas diferentes e simultâneas de analisar como as palavras da frase se relacionam umas com as outras (ex: uma cabeça pode focar na gramática, outra no sujeito da ação).

### 4. Processamento (Inferência)
* **`embedded = embed(tokens)`**: Converte os IDs em vetores isolados. O formato agora é `[4, 1, 16]`.
* **`output = tf_layer(embedded)`**: Passa os vetores pelo Transformer.
* **O Grande Segredo**: O formato de saída continua sendo `[4, 1, 16]`, mas os números lá dentro mudaram completamente. Antes de passar pelo Transformer, o vetor da palavra "powerful" significava apenas "poderoso". Após o Transformer, o vetor da palavra "powerful" foi modificado matematicamente para conter o **contexto** de que estamos falando sobre "Deep learning".

In [9]:
# Creating context-aware embeddings for a 4-word sentence.
sentence = "Deep learning is powerful"
vocab = {"Deep": 0, "learning": 1, "is": 2, "powerful": 3}
tokens = torch.tensor([vocab[w] for w in sentence.split()]).unsqueeze(1) # [Seq_Len, Batch]

embed = nn.Embedding(num_embeddings=10, embedding_dim=16)
tf_layer = nn.TransformerEncoderLayer(d_model=16, nhead=4)

with torch.no_grad():
    embedded = embed(tokens)
    output = tf_layer(embedded)

print("\n--- TRANSFORMER TEXT TEST ---")
print(f"Input Sentence: '{sentence}'")
print(f"Token IDs: {tokens.flatten().tolist()}")
print(f"Output Context Vectors Shape: {output.shape}") # [4, 1, 16]


--- TRANSFORMER TEXT TEST ---
Input Sentence: 'Deep learning is powerful'
Token IDs: [0, 1, 2, 3]
Output Context Vectors Shape: torch.Size([4, 1, 16])


# Compressão e Reconstrução de Imagens com Autoencoders

Este bloco demonstra o funcionamento de um **Autoencoder**, uma rede neural dividida em duas partes: um **Encoder** (que comprime os dados) e um **Decoder** (que tenta reconstruir os dados originais a partir da versão comprimida). É a base para redução de dimensionalidade e geração de novas imagens.



### 1. Preparando a Imagem Original
* **`mnist_test = datasets.MNIST(...)`**: Carregamos novamente o dataset MNIST (dígitos escritos à mão).
* **`real_digit, _ = mnist_test[0]`**: Pegamos a primeira imagem do teste (que costuma ser um número '7'). Ignoramos o rótulo (`_`) porque Autoencoders são modelos não-supervisionados; eles não se importam com a classe da imagem, apenas com os pixels dela.
* **`real_digit.view(-1, 784)`**: Achatamos a imagem de 28x28 pixels para um vetor de 784 números, preparando-a para as camadas lineares. O `-1` diz ao PyTorch para deduzir automaticamente o tamanho do lote (*batch*).

### 2. O Codificador (Encoder)
* **`encoder = nn.Linear(784, 20)`**: Esta é a fase de compressão (o "gargalo" da rede). Pegamos os 784 pixels originais e os forçamos a caber em um vetor de apenas 20 números. Essa representação comprimida é chamada de **Espaço Latente** (*Latent Space*).

### 3. O Decodificador (Decoder)
* **`decoder = nn.Sequential(...)`**: Esta rede faz o caminho inverso.
    * **`nn.Linear(20, 400)` e `nn.ReLU()`**: Pega o vetor comprimido de 20 números e começa a expandi-lo para 400 neurônios.
    * **`nn.Linear(400, 784)`**: Expande novamente para os 784 pixels originais.
    * **`nn.Sigmoid()`**: Como os pixels originais da imagem foram convertidos pelo `ToTensor()` para valores entre 0 e 1, a função Sigmoid garante que a saída da nossa rede também fique restrita a esse mesmo intervalo.

### 4. Executando a Compressão e Reconstrução
* **`latent_vector = encoder(...)`**: Transforma o dígito '7' em um resumo de 20 números.
* **`reconstruction = decoder(latent_vector).view(1, 28, 28)`**: Pega o resumo de 20 números, tenta reconstruir os 784 pixels e usa o `view` para remodelar o vetor de volta ao formato de grade 2D (imagem de 28x28).
* Os **prints** finais somam os valores dos pixels para comparar a imagem original com a reconstruída.

> **💡 Nota:** Como o modelo não foi treinado, o "resumo" criado pelo Encoder é aleatório, e a reconstrução feita pelo Decoder parecerá apenas ruído (chuvisco de TV). Para que ele aprenda a reconstruir o '7' perfeitamente, precisamos treiná-lo comparando a saída com a entrada.

In [ ]:
# Compressing a digit to 20 numbers and rebuilding it.
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
real_digit, _ = mnist_test[0] # A '7'
real_digit_flat = real_digit.view(-1, 784)

encoder = nn.Linear(784, 20)
decoder = nn.Sequential(nn.Linear(20, 400), nn.ReLU(), nn.Linear(400, 784), nn.Sigmoid())

with torch.no_grad():
    latent_vector = encoder(real_digit_flat)
    reconstruction = decoder(latent_vector).view(1, 28, 28)

print("\n--- VAE MNIST RECONSTRUCTION TEST ---")
print(f"Input image pixels: {real_digit.sum().item():.2f}")
print(f"Reconstructed pixels: {reconstruction.sum().item():.2f}")


--- VAE MNIST RECONSTRUCTION TEST ---
Input image pixels: 72.37
Reconstructed pixels: 392.77
